<a href="https://colab.research.google.com/github/haytamelouarrat00/Cone_Detection_FS/blob/master/Cone_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="J09U3OiEmcboSrbUKAgf")
project = rf.workspace("fsbdriverless-xi0fv").project("fsai-11-03-2024")
version = project.version(1)
dataset = version.download("yolov11")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to FSAI-11/03/2024-1 in yolov11:: 100%|██████████| 47504/47504 [00:07<00:00, 6517.96it/s]


In [ ]:
!pip install ultralytics

In [ ]:
import os
from ultralytics import YOLO
import torch

In [ ]:

# Configuration
DATASET_PATH = '/content/FSAI-11/03/2024-1'
MODEL_PATH = 'yolo11s.pt'
EPOCHS = 100
BATCH_SIZE = 16
IMAGE_SIZE = 640
NUM_CLASSES = 5


print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")

model = YOLO(MODEL_PATH)

# Training
results = model.train(
    data=os.path.join(DATASET_PATH, 'data.yaml'),
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    workers=8,
    device=0,
    plots=True,
    project='runs/detect',
    name='custom_training',
    resume=False,

    # Advanced hyperparameters
    lr0=0.01,
    lrf=0.1,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    close_mosaic=10
)

# Validation
validation_results = model.val()

# Export model
model.export(format='onnx')

In [ ]:
import onnxruntime as ort
from PIL import Image


onnx_model_path = "runs/detect/custom_training/weights/best.onnx"
ort_session = ort.InferenceSession(onnx_model_path)


def preprocess_image(image_path, img_size=640):
    image = Image.open(image_path).convert('RGB')
    image = image.resize((img_size, img_size))
    img_array = np.array(image).astype('float32') / 255.0
    img_array = img_array.transpose(2, 0, 1)  # HWC to CHW
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

image_path = "test1.png"
input_data = preprocess_image(image_path)

outputs = ort_session.run(None, {ort_session.get_inputs()[0].name: input_data})
print(outputs)


In [ ]:
"""from ultralytics import YOLO
import cv2
import numpy as np

def run_inference(model_path, image_path, conf_thresh=0.25):
    """"""
    Run YOLO inference on a single image

    Args:
        model_path: Path to trained YOLO model (.pt file)
        image_path: Path to test image
        conf_thresh: Confidence threshold for detections
    """"""
    model = YOLO(model_path)
    
    results = model(image_path, conf=conf_thresh)[0]

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Plot results
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        conf = box.conf[0]
        cls = int(box.cls[0])

        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        label = f'{results.names[cls]} {conf:.2f}'
        cv2.putText(image, label, (x1, y1 - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    output_path = 'result.jpg'
    cv2.imwrite(output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    print(f"Results saved to {output_path}")

    # Print detections
    print("\nDetections:")
    for box in results.boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        print(f"Class: {results.names[cls]}, Confidence: {conf:.2f}")

model_path = "runs/detect/custom_training/weights/best.pt"
image_path = "test1.png"

run_inference(model_path, image_path)
"""

In [ ]:
import cv2
import time
import numpy as np
from pathlib import Path

def process_video(model_path, video_path, output_path=None, conf_thresh=0.25):
    """
    Process video with YOLO model

    Args:
        model_path: Path to trained YOLO model (.pt file)
        video_path: Path to input video
        output_path: Path for output video (optional)
        conf_thresh: Confidence threshold for detections
    """
    # Load the model
    model = YOLO(model_path)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if output_path is None:
        output_path = str(Path(video_path).with_name(f"{Path(video_path).stem}_detected.mp4"))

    out = cv2.VideoWriter(output_path,
                         cv2.VideoWriter_fourcc(*'mp4v'),
                         fps,
                         (frame_width, frame_height))

    frame_count = 0
    start_time = time.time()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1


        results = model(frame, conf=conf_thresh)[0]

        # Draw boxes and labels
        annotated_frame = results.plot()

        # Add processing info
        elapsed_time = time.time() - start_time
        fps_current = frame_count / elapsed_time
        cv2.putText(annotated_frame, f'FPS: {fps_current:.1f}',
                   (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        
        out.write(annotated_frame)
        if frame_count % 30 == 0:  # Update every 30 frames
            progress = (frame_count / total_frames) * 100
            print(f"Progress: {progress:.1f}% ({frame_count}/{total_frames} frames)")

    cap.release()
    out.release()
    print(f"\nProcessing complete! Output saved to: {output_path}")
    print(f"Average FPS: {frame_count / elapsed_time:.1f}")

def process_webcam(model_path, conf_thresh=0.25):
    """
    Real-time webcam detection

    Args:
        model_path: Path to trained YOLO model (.pt file)
        conf_thresh: Confidence threshold for detections
    """
    
    model = YOLO(model_path)

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame, conf=conf_thresh)[0]
        annotated_frame = results.plot()
        cv2.imshow('YOLO Detection', annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
if __name__ == "__main__":
    model_path = "runs/detect/custom_training/weights/best.pt"

    # For video file:
    video_path = "testvid.mp4"
    process_video(model_path, video_path)

    # For webcam:
    # process_webcam(model_path)

Fine-tuning

In [ ]:

from ultralytics import YOLO

model = YOLO('res/weights/best.pt')

confidence_threshold = 0.5

# Run predictions
results = model.predict(
    source='testvid3.mp4',
    conf=confidence_threshold,
    save=True,
    save_txt=True,
    imgsz=640
)

# Evaluate results (if available metrics are provided)
print(results)
